In [1]:
import sys
sys.path.insert(0, './src')  # 或者绝对路径：/full/path/to/your_project
from src.xmcdata import *
from gen_model import *

In [ ]:
# Example configurations for different models
data_dir = "./xmc-base/"
dataset_name = "eurlex-4k/"
dataset_dir = data_dir+dataset_name
model_names = [
    "google/flan-t5-base",  # T5-base
    "google/flan-t5-large",  # T5-large
    "google/flan-t5-xl",     # T5-xl
    "facebook/bart-base",    # BART
    "google/pegasus-xsum"    # Pegasus
]
# T5-FLAN Configuration
t5_config = T2TConfig(
    model_name=model_names[0],  # Change to other models as needed
    dataset_dir=dataset_dir,
    output_dir=dataset_dir+"outputs/"+model_names[0].split("/")[-1],
    prompt="Summarize this document by unstemmed keyphrases:",
    #task_prefix="classify:",  # T5-style task prefix
    num_epochs=5,
    batch_size=16, # base-16, large-4, xl-2? for 24GB
    learning_rate= 2e-5,
    max_input_length=512,
    max_output_length=128,
    strict_tokenization=True,
    fp16=False, #t5 cannot use fp16
    bf16= True,
)

# BART Configuration
bart_config = T2TConfig(
    model_name=model_names[3],  # Change to other models as needed
    dataset_dir=dataset_dir,
    output_dir=dataset_dir+"outputs/"+model_names[3].split("/")[-1],
    prompt="Summarize this document by unstemmed keyphrases:",
    num_epochs=3,
    batch_size=4,
    max_input_length=512,
    max_output_length=128,
    learning_rate= 2e-5,
    fp16=True,
    bf16= False,
    strict_tokenization=True  # Apply T5-style strict standards to BART
)

# Pegasus Configuration
pegasus_config = T2TConfig(
    model_name=model_names[4],  # Change to other models as needed
    dataset_dir=dataset_dir,
    output_dir=dataset_dir+"outputs/"+model_names[4].split("/")[-1],
    prompt="Summarize this document by unstemmed keyphrases:",
    num_epochs=3,
    batch_size=4,  # Smaller batch for Pegasus
    max_input_length=512,
    max_output_length=128,
    learning_rate= 2e-5,
    fp16=True,
    bf16= False,
    strict_tokenization=True  # Apply T5-style strict standards to Pegasus
)

In [3]:
# Choose configuration
config = t5_config  # Change to bart_config or pegasus_config as needed

# Initialize model
model = UniversalT2TModel(config)

# Save configuration
model.save_config()

# Train the model (uncomment to train)
model.train()

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loaded T5 tokenizer: google/flan-t5-base
Loading T5 model: google/flan-t5-base
Model loaded on cuda
Model type: T5
Model parameters: 247,577,856
Configuration saved to xmc-base/eurlex-4k/outputs/flan-t5-base/config.json
Starting training with T5-style strict standards...
Loading tokenized data from cache:./xmc-base/eurlex-4k//train.pt
Loaded 15449 samples from cache
Loading tokenized data from cache:./xmc-base/eurlex-4k//test.pt
Loaded 3865 samples from cache


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
100,4.317400
200,3.692800
300,3.025400
400,2.714800
500,2.511600
600,2.363100
700,2.287000
800,2.176100
900,2.094100
1000,2.056600


Training completed. Model saved to ./xmc-base/eurlex-4k/outputs/flan-t5-base


T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [4]:
# Generate predictions for test set
print("Generating predictions with T5-style strict processing...")
predictions = model.predict_test_set(output_path=dataset_dir)

print(f"Generated {len(predictions)} predictions using {model.tokenizer.model_type.upper()}")
print("Sample predictions:")
for i, pred in enumerate(predictions[:3]):
    print(f"{i+1}: {pred}")

Generating predictions with T5-style strict processing...
Loading tokenized data from cache:./xmc-base/eurlex-4k//train.pt


Loaded 15449 samples from cache
Loading tokenized data from cache:./xmc-base/eurlex-4k//test.pt
Loaded 3865 samples from cache


Generating with T5 Standards: 100%|██████████| 242/242 [05:04<00:00,  1.26s/it]

Generated 3865 predictions using T5
Sample predictions:
1: beef,france,marketing aid,intervention stock,intervention product.
2: ec commission,ec aid to industry,ec aid to industry,shipbuilding industry,sale price.
3: common organisation of markets,export refund,fishery product,export refund.
